In [34]:
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz

You should consider upgrading via the '/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz (532.3 MB)
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [49]:
from collections import defaultdict
import os
from pprint import pprint
import spacy
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from typing import List, Tuple

In [36]:
def get_tag_counts(dir_path):
    filenames = [filename for filename in os.listdir(dir_path) if filename.endswith(".ann")]

    entity_tags = defaultdict(set)
    entity_tag_counts = defaultdict(int)
    entity_tag_unique_counts = defaultdict(int)
    relationship_tag_counts = defaultdict(int)

    for filename in filenames:
        with open(dir_path + filename) as f:
            text = f.read()

        lines = [line.split("\t") for line in text.split("\n")[:-1]]

        for line in lines:
            tag = line[1].split(" ")[0]

            if line[0].startswith("T"):    
                entity_tag_counts[tag] += 1
                if line[2] not in entity_tags[tag]:
                    entity_tags[tag].add(line[2])
                    entity_tag_unique_counts[tag] += 1
            elif line[0].startswith("R"):
                relationship_tag_counts[tag] += 1
    
    return entity_tag_counts, entity_tag_unique_counts, relationship_tag_counts

In [38]:
entity_tag_counts, entity_tag_unique_counts, relationship_tag_counts = get_tag_counts("../../data/training_20180910/")

print("entity_tag_counts")
pprint(entity_tag_counts)
print("entity_tag_unique_counts")
pprint(entity_tag_unique_counts)
print("relationship_tag_counts")
pprint(relationship_tag_counts)

entity_tag_counts
defaultdict(<class 'int'>,
            {'ADE': 959,
             'Dosage': 4221,
             'Drug': 16225,
             'Duration': 592,
             'Form': 6651,
             'Frequency': 6281,
             'Reason': 3855,
             'Route': 5476,
             'Strength': 6691})
entity_tag_unique_counts
defaultdict(<class 'int'>,
            {'ADE': 549,
             'Dosage': 433,
             'Drug': 3186,
             'Duration': 232,
             'Form': 326,
             'Frequency': 721,
             'Reason': 1772,
             'Route': 189,
             'Strength': 871})
relationship_tag_counts
defaultdict(<class 'int'>,
            {'ADE-Drug': 1107,
             'Dosage-Drug': 4225,
             'Duration-Drug': 643,
             'Form-Drug': 6654,
             'Frequency-Drug': 6310,
             'Reason-Drug': 5169,
             'Route-Drug': 5538,
             'Strength-Drug': 6702})


In [39]:
entity_tag_counts, entity_tag_unique_counts, relationship_tag_counts = get_tag_counts("../../data/test/")

print("entity_tag_counts")
pprint(entity_tag_counts)
print("entity_tag_unique_counts")
pprint(entity_tag_unique_counts)
print("relationship_tag_counts")
pprint(relationship_tag_counts)

entity_tag_counts
defaultdict(<class 'int'>,
            {'ADE': 625,
             'Dosage': 2681,
             'Drug': 10575,
             'Duration': 378,
             'Form': 4359,
             'Frequency': 4012,
             'Reason': 2545,
             'Route': 3513,
             'Strength': 4230})
entity_tag_unique_counts
defaultdict(<class 'int'>,
            {'ADE': 383,
             'Dosage': 297,
             'Drug': 2407,
             'Duration': 181,
             'Form': 284,
             'Frequency': 532,
             'Reason': 1256,
             'Route': 175,
             'Strength': 677})
relationship_tag_counts
defaultdict(<class 'int'>,
            {'ADE-Drug': 733,
             'Dosage-Drug': 2695,
             'Duration-Drug': 426,
             'Form-Drug': 4374,
             'Frequency-Drug': 4034,
             'Reason-Drug': 3410,
             'Route-Drug': 3546,
             'Strength-Drug': 4244})


In [40]:
nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")

/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [127]:
dir_path = "../../data/training_20180910/"
filenames = [filename for filename in os.listdir(dir_path) if filename.endswith(".ann")]

drug_entity_counts = defaultdict(int)
ade_entity_counts = defaultdict(int)    

for i, filename in enumerate(filenames):
    with open(dir_path + filename) as f:
        text = f.read() 

    lines = [line.split("\t") for line in text.split("\n")[:-1]]

    for line in lines:
        if line[0].startswith("T"):
            if line[1].split(" ")[0] == "Drug":
                doc = nlp(line[2])
                drug_entity_counts[len(doc.ents)] += 1
            elif line[1].split(" ")[0] == "ADE":
                doc = nlp(line[2])
                ade_entity_counts[len(doc.ents)] += 1

    if (i + 1) % 10 == 0:
        print(f"Completed for {i + 1} files...")

print("drug_entity_counts")
print(drug_entity_counts)
print("ade_entity_counts")
print(ade_entity_counts)

Completed for 10 files...
Completed for 20 files...
Completed for 30 files...
Completed for 40 files...
Completed for 50 files...
Completed for 60 files...
Completed for 70 files...
Completed for 80 files...
Completed for 90 files...
Completed for 100 files...
Completed for 110 files...
Completed for 120 files...
Completed for 130 files...
Completed for 140 files...
Completed for 150 files...
Completed for 160 files...
Completed for 170 files...
Completed for 180 files...
Completed for 190 files...
Completed for 200 files...
Completed for 210 files...
Completed for 220 files...
Completed for 230 files...
Completed for 240 files...
Completed for 250 files...
Completed for 260 files...
Completed for 270 files...
Completed for 280 files...
Completed for 290 files...
Completed for 300 files...
drug_entity_counts
defaultdict(<class 'int'>, {1: 14908, 2: 712, 0: 568, 4: 4, 3: 33})
ade_entity_counts
defaultdict(<class 'int'>, {2: 127, 1: 801, 3: 17, 0: 13, 4: 1})


In [128]:
dir_path = "../../data/test/"
filenames = [filename for filename in os.listdir(dir_path) if filename.endswith(".ann")]

drug_entity_counts = defaultdict(int)
ade_entity_counts = defaultdict(int)    

for i, filename in enumerate(filenames):
    with open(dir_path + filename) as f:
        text = f.read() 

    lines = [line.split("\t") for line in text.split("\n")[:-1]]

    for line in lines:
        if line[0].startswith("T"):
            if line[1].split(" ")[0] == "Drug":
                doc = nlp(line[2])
                drug_entity_counts[len(doc.ents)] += 1
            elif line[1].split(" ")[0] == "ADE":
                doc = nlp(line[2])
                ade_entity_counts[len(doc.ents)] += 1

    if (i + 1) % 10 == 0:
        print(f"Completed for {i + 1} files...")

print("drug_entity_counts")
print(drug_entity_counts)
print("ade_entity_counts")
print(ade_entity_counts)

Completed for 10 files...
Completed for 20 files...
Completed for 30 files...
Completed for 40 files...
Completed for 50 files...
Completed for 60 files...
Completed for 70 files...
Completed for 80 files...
Completed for 90 files...
Completed for 100 files...
Completed for 110 files...
Completed for 120 files...
Completed for 130 files...
Completed for 140 files...
Completed for 150 files...
Completed for 160 files...
Completed for 170 files...
Completed for 180 files...
Completed for 190 files...
Completed for 200 files...
drug_entity_counts
defaultdict(<class 'int'>, {1: 9728, 2: 455, 0: 368, 3: 20, 4: 4})
ade_entity_counts
defaultdict(<class 'int'>, {1: 510, 2: 88, 3: 12, 0: 12, 4: 2, 6: 1})


In [52]:
class Augmentor:
    def __init__(self, model_name: str="en_core_sci_lg") -> None:
        self.nlp = spacy.load(model_name)
        self.nlp.add_pipe("abbreviation_detector")
        self.nlp.add_pipe(
            "scispacy_linker",
            config={"resolve_abbreviations": True, "linker_name": "umls"}
        )
        self.linker = self.nlp.get_pipe("scispacy_linker")

    def augment(
            self,
            text: str, 
            s1: int,
            e1: int,
            s2: int, 
            e2: int, 
            aug_factor: int=10, 
            score_threshold: float=0.95
        ) -> List[Tuple[str, int, int, int, int]]:
        """
        Args:
            text (str): Input text sample
            s1 (int): Start index of first entity in `text`
            e1 (int): End index of first entity in `text`
            s2 (int): Start index of second entity in `text` (it is assumed that
              `e1` < `s2`)
            e2 (int): End index of second entity in `text`
            aug_factor (int, optional): No. of augmented samples to generate for
              the input sample. It is assumed that `aug_factor` is a positive
              integer. Defaults to 10.
            score_threshold (float, optional): Score threshold linked entities 
              must satisfy to be used for augmentation. Defaults to 0.95.

        Returns:
            List[Tuple[str, int, int, int, int]]: List of augmented samples
        """
        result = [(text, s1, e1, s2, e2)]

        if aug_factor == 1:
            return result
        
        doc = self.nlp(text)
        for ent in doc.ents:
            # Find start and end indices of ent in text
            s0 = text.find(ent.text) # TODO: Address duplicates issue
            e0 = s0 + len(ent.text)
            prefix = text[:s0] # Text before entity
            suffix = text[e0:] # Text after entity

            for cui, score in ent._.kb_ents:
                if score_threshold <= score:
                    for alias in set(self.linker.kb.cui_to_entity[cui].aliases):
                        # TODO: Impose constraints on alias
                        aug_text = prefix + alias + suffix
                        d = len(alias) - (e0 - s0)

                        if e0 < s1:
                            result.append(
                                (aug_text, s1 + d, e1 + d, s2 + d, e2 + d))
                        elif s0 == s1 and e0 == e1:
                            result.append(
                                (aug_text, s1, s1 + len(alias), s2 + d, e2 + d))
                        elif e1 < s0 and e0 < s2:
                            result.append((aug_text, s1, e1, s2 + d, e2 + d))
                        elif s0 == s2 and e0 == e2:
                            result.append((aug_text, s1, e1, s2, s2 + len(alias)))
                        elif e2 < s0:
                            result.append((aug_text, s1, e1, s2, e2))

                        # Return if desired augmentation ratio is achieved
                        if aug_factor <= len(result):
                            return result

        return result

In [53]:
augmentor = Augmentor()

/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [54]:
augmentor.augment(text, s1, e1, s2, e2)

/Users/rohan/Projects/biomedical-ner-relationship-classification/venv/lib/python3.8/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


[('sembryoplastic angioneural epithelial lesion\nwith features of an oligodendroglioma who was started on\nDilantin postoperatively for seizure prophylaxis and was\nsubsequently developed eye discharge and was seen by an\noptometrist who treated it with sulfate ophthalmic drops.\nThe patient then developed ',
  102,
  110,
  182,
  195),
 ('sembryoplastic angioneural epithelial lesion\nwith Feature of an oligodendroglioma who was started on\nDilantin postoperatively for seizure prophylaxis and was\nsubsequently developed eye discharge and was seen by an\noptometrist who treated it with sulfate ophthalmic drops.\nThe patient then developed ',
  101,
  109,
  181,
  194),
 ('sembryoplastic angioneural epithelial lesion\nwith Characteristics of an oligodendroglioma who was started on\nDilantin postoperatively for seizure prophylaxis and was\nsubsequently developed eye discharge and was seen by an\noptometrist who treated it with sulfate ophthalmic drops.\nThe patient then developed ',
  1